In [2]:
# 12/17(수) 10:30

# Output Parser
**Output Parser**: 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환.
   - JSON(JavaScript Object Notation): 데이터 구조를 표현하기 위한 경량 포맷.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환.
   - Pydantic: 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환. 거기에서 응답 문자열만 추출해서 반환.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 단순 질의용 prompt 생성 - from_template() 메소드 이용. (PromptTemplate용)
prompt = ChatPromptTemplate.from_template(
    template="한국의 {topic}에 관련된 속담 {count}개를 알려줘. 목록 형식으로 출력해줘."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()
# prompt -> model -> parser

query = prompt.invoke({"topic":"호랑이", "count":3})
res = model.invoke(query)
final_res = parser.invoke(res)

In [5]:
res

AIMessage(content='1. 호랑이도 제 말 하면 온다. — 누군가를 말하고 있으면 그 사람이 뜻밖에 나타난다는 뜻(영어의 "speak of the devil"과 유사).\n\n2. 호랑이에게 물려가도 정신만 차리면 산다. — 큰 위험에 처해도 침착하면 살 길이 있다는 뜻(위기 상황에서 냉정함의 중요성).\n\n3. 호랑이 없는 굴에 여우가 왕. — 강한 존재가 없으면 약한 자가 그 자리를 차지하게 된다는 뜻(권력 공백 시의 상황 설명).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 976, 'prompt_tokens': 30, 'total_tokens': 1006, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CncEcYFXwGpdZGqMzmWScMEc8FhBH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2a4a-72e9-7722-bce2-774fcfa8ff85-0', usage_metadata={'input_tokens': 30, 'output_tokens': 976, 'total_tokens': 1006, 'input_token_details': {'audio': 0, 'cache_read'

In [6]:
print(final_res)

1. 호랑이도 제 말 하면 온다. — 누군가를 말하고 있으면 그 사람이 뜻밖에 나타난다는 뜻(영어의 "speak of the devil"과 유사).

2. 호랑이에게 물려가도 정신만 차리면 산다. — 큰 위험에 처해도 침착하면 살 길이 있다는 뜻(위기 상황에서 냉정함의 중요성).

3. 호랑이 없는 굴에 여우가 왕. — 강한 존재가 없으면 약한 자가 그 자리를 차지하게 된다는 뜻(권력 공백 시의 상황 설명).


## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환.
  - "a,b,c" => ['a','b','c']

In [7]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
txt = "이순신, 유관순, 안중근, 강감찬, 세종대왕"
txt = "요청하신 이름은 이순신과 유관순과 안중근입니다."
r1 = parser.parse(txt)
r2 = parser.invoke(txt)
print(type(r1), type(r2))

<class 'list'> <class 'list'>


In [8]:
print(r1)
print(r2)

['요청하신 이름은 이순신과 유관순과 안중근입니다.']
['요청하신 이름은 이순신과 유관순과 안중근입니다.']


In [9]:
# 이 output parser에 맞는 출력을 모델이 하도록 하는 지시문으로 프롬프트에 추가한다. 
parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [10]:
parser = CommaSeparatedListOutputParser()
prompt = ChatPromptTemplate(
    messages=[
        {"role":"system", "content": "Output Format:{format_instruction}"},
        {"role":"user", "content":"{subject}에 대해 다섯가지를 나열해주세요."}
    ],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
# partial_variables={"input_variable": 넣을 값,} => input_variable의 값을 Prompt Template을 만들면서 넣어준다 
# (invoke 시 넣는 것이 아니라 생성 시 설정.)
model = ChatOpenAI(model="gpt-5-mini")

query = prompt.invoke({"subject": "자동차 종류"})
res = model.invoke(query)

In [11]:
res.content
final_res = parser.invoke(res)
final_res

['세단', 'SUV', '해치백', '쿠페', '컨버터블']

In [12]:
# 질의 -> (Prompt) -> query -> (Model) -> res -> (Parser) -> 최종 답변
# chain: 위 실행 흐름을 자동화 (pipeline)
chain = prompt | model | parser
res = chain.invoke({"subject":"오픈 소스 LLM 모델 이름"})
print(res)

['LLaMA 2', 'MPT-7B', 'Falcon-40B', 'BLOOM', 'Vicuna']


## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환.

In [13]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
print(parser.get_format_instructions())

txt = '{"name":"홍길동", "age":20}'
r = parser.invoke(txt)
print(type(r))
r

Return a JSON object.
<class 'dict'>


{'name': '홍길동', 'age': 20}

In [ ]:
parser = JsonOutputParser()
# parser = JsonOutputParser(pydantic_object=PersonInfo)

prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [18]:
final_res = parser.invoke(res)

In [21]:
print(type(res.content), type(final_res))

<class 'str'> <class 'dict'>


In [20]:
res.content

'{\n  "이름": "이순신(李舜臣)",\n  "생몰": {\n    "출생연도": "1545년",\n    "사망일": "1598년 12월 16일 (노량해전에서 전사)"\n  },\n  "한줄소개": "조선 중기의 무신이자 명장. 임진왜란(1592–1598) 동안 해상에서 수많은 승리를 거두며 국난을 막아낸 장군으로, 충무공(忠武公)으로 숭앙됨.",\n  "주요업적": [\n    "임진왜란 기간 조선 수군을 지휘하여 연전연승을 거둠(해전에서 패한 적이 거의 없음).",\n    "한산도 대첩(1592) 등에서 학익진(鶴翼陣) 등 전술로 일본 함대를 격파하여 해상 우위를 확보.",\n    "명량해전(1597)에서 극소수의 함대로 압도적 수적 우위의 적을 상대로 대승을 거둠.",\n    "거북선의 전투적 운용 및 해군 전술의 발전에 기여(거북선의 기원과 설계에는 논란이 있으나 실전 운용과 개량에 중요한 역할).",\n    "해상 보급선 차단과 적의 해상 작전 봉쇄를 통해 육군 작전 지원 및 국토 방위에 결정적 기여."\n  ],\n  "주요해전(일부)": [\n    {\n      "전투명": "옥포해전",\n      "연도": "1592년",\n      "의의": "임진왜란 초기에 거둔 최초의 해전 승리 중 하나로 수군의 사기 진작."\n    },\n    {\n      "전투명": "한산도 대첩",\n      "연도": "1592년",\n      "의의": "학익진을 활용해 일본 수군의 해상 보급로를 차단하여 전략적 우위 확보."\n    },\n    {\n      "전투명": "명량해전",\n      "연도": "1597년",\n      "의의": "극소수의 전함으로 수적 우세의 일본 함대를 상대로 대승을 거둠(이순신의 대표적 역전극)."\n    },\n    {\n      "전투명": "노량해전",\n      "연도": "1598년",\n      "의의": "이 전투에서 이순신 장군이 전사함. 임진왜란의 마지막 해전 중

In [19]:
print(res.content)

{
  "이름": "이순신(李舜臣)",
  "생몰": {
    "출생연도": "1545년",
    "사망일": "1598년 12월 16일 (노량해전에서 전사)"
  },
  "한줄소개": "조선 중기의 무신이자 명장. 임진왜란(1592–1598) 동안 해상에서 수많은 승리를 거두며 국난을 막아낸 장군으로, 충무공(忠武公)으로 숭앙됨.",
  "주요업적": [
    "임진왜란 기간 조선 수군을 지휘하여 연전연승을 거둠(해전에서 패한 적이 거의 없음).",
    "한산도 대첩(1592) 등에서 학익진(鶴翼陣) 등 전술로 일본 함대를 격파하여 해상 우위를 확보.",
    "명량해전(1597)에서 극소수의 함대로 압도적 수적 우위의 적을 상대로 대승을 거둠.",
    "거북선의 전투적 운용 및 해군 전술의 발전에 기여(거북선의 기원과 설계에는 논란이 있으나 실전 운용과 개량에 중요한 역할).",
    "해상 보급선 차단과 적의 해상 작전 봉쇄를 통해 육군 작전 지원 및 국토 방위에 결정적 기여."
  ],
  "주요해전(일부)": [
    {
      "전투명": "옥포해전",
      "연도": "1592년",
      "의의": "임진왜란 초기에 거둔 최초의 해전 승리 중 하나로 수군의 사기 진작."
    },
    {
      "전투명": "한산도 대첩",
      "연도": "1592년",
      "의의": "학익진을 활용해 일본 수군의 해상 보급로를 차단하여 전략적 우위 확보."
    },
    {
      "전투명": "명량해전",
      "연도": "1597년",
      "의의": "극소수의 전함으로 수적 우세의 일본 함대를 상대로 대승을 거둠(이순신의 대표적 역전극)."
    },
    {
      "전투명": "노량해전",
      "연도": "1598년",
      "의의": "이 전투에서 이순신 장군이 전사함. 임진왜란의 마지막 해전 중 하나."
    }
  ],
  "전술·기술": [
    

In [ ]:
########## 이 아래로 강사님 거 보고 채우기 #########

In [ ]:
# JSON 응답 스키마 (구조 설계) 정의 -> Pydantic
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")

# 변수: Key
# Field.description: 변수에 넣을 내용.

In [ ]:
parser2 = JsonOutputParser(pydantic_object=PersonInfo)
print(parser2.get_format_instructions())

In [ ]:
# parser = JsonOutputParser()
parser = JsonOutputParser(pydantic_object=PersonInfo)

prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [ ]:
final_res = parser.invoke(res)

In [ ]:
print(type(res.content), type(final_res))
final_res

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [1]:
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description='조회한 사람의 이름')
    yob: int = Field(description='조회한 사람이 태어난 년도')
    yod: int = Field(description='조회한 사람이 사망한 년도')
    profile: str = Field(description='조회한 사람의 주요 업적')

In [2]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=PersonInfo)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "조회한 사람의 이름", "title": "Name", "type": "string"}, "yob": {"description": "조회한 사람이 태어난 년도", "title": "Yob", "type": "integer"}, "yod": {"description": "조회한 사람이 사망한 년도", "title": "Yod", "type": "integer"}, "profile": {"description": "조회한 사람의 주요 업적", "title": "Profile", "type": "string"}}, "required": ["name", "yob", "yod", "profile"]}
```


In [ ]:
from langchain_core.output_parsers import PydanticParser

parser = PydanticOutputParser(pydantic_object=PersonInfo)
prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [ ]:
final_res = parser.invoke(res)
type(final_res)

In [ ]:
final_res

In [ ]:
final_res.name
final_res.profile

# LLM모델에 출력 형식을 설정

- ChatModel객체의 `with_structured_output(pydantic.BaseModel)` 을 이용해 모델의 출력 형식을 모델 자체에 추가할 수있다.
- `OutputParser`는 모델의 출력 결과를 받아서 형식을 변경해 준다. 그래서 Chain에 탈/부착을 통해 형식을 적용하거나 적용하지 않는 것을 자유롭게 할 수있다.
- 모델의 출력 결과를 항상 일정하게 할 경우에는 아예 **모델에 출력 형식을 설정할 수 있다.**

In [4]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# 출력 schema  정의
class OutputSchema(BaseModel):
    is_real_person: bool = Field(..., description="요청한 인물이 실존 인물인지 여부. True: 실존인물,  False: 실존인물이 아님")
    description: str = Field(..., description="요청한 인물에 대한 설명")

model = ChatOpenAI(model='gpt-5-mini')
res = model.invoke("이순신 장군에 대해 설명해줘.")

In [5]:
print(type(res))
print(res.content)

<class 'langchain_core.messages.ai.AIMessage'>
이순신 장군(李舜臣, 1545–1598)은 조선 중기의 대표적인 무신이자 해군 장군으로, 임진왜란(1592–1598) 때 탁월한 해전 지휘로 왜군의 해상 보급로를 차단해 국난을 극복하는 데 결정적 역할을 한 인물입니다. 주요 내용만 간단히 정리하면 다음과 같습니다.

- 생애 개요: 1545년 출생. 무과에 급제해 군인으로 출발하여 해군 지휘관으로 활동했습니다. 권력 다툼으로 한때 좌천되기도 했다가 임진왜란 발발 후 다시 중용되었습니다.
- 주요 계급과 직책: 삼도수군통제사(三道水軍統制使) 등 중요한 해군 지휘관을 역임했습니다. 전후에 충무공(忠武公)이라는 시호가 내려졌습니다.
- 주요 전과
  - 옥포 해전(1592) 등 초기 해전에서 연승을 거두어 일본의 남해·서해 진출을 저지했습니다.
  - 한산도 대첩(1592): 학익진(鶴翼陣) 등의 전술로 큰 승리를 거두어 일본 수군의 활동을 봉쇄했습니다.
  - 명량 해전(1597): 13척의 배로 300여 척의 일본 함대를 상대로 승리한 기적적 전투(명량대첩)로 유명합니다. 좁은 해협과 조류를 이용한 탁월한 전술의 사례입니다.
  - 노량 해전(1598): 전쟁 마지막 해전에서 적과 교전 중 전사했습니다.
- 군사적 업적 및 전술
  - 철저한 훈련과 규율, 정보·기상·지형을 활용한 전술 운용으로 적을 압도했습니다.
  - 거북선(거북선의 사용·형태에 관해 연구·논쟁이 있으나, 이순신이 화포를 효과적으로 활용하고 전열을 조직해 해전에서 우위를 점했다는 점은 널리 인정됩니다.
- 기록과 유산
  - 난중일기(亂中日記): 전쟁 기간 직접 쓴 일기로, 병법·지휘관의 인간적인 고뇌·현장 기록이 담겨 있어 역사적 자료로 매우 중요합니다.
  - 한국에서는 국민적 영웅으로 기려지며 충무공 이순신으로 불리고, 여러 기념사업과 사당·동상·박물관 등으로 기념됩니다.

간단히 말해, 이순신 장군은 뛰어난 전략과 리더십으로 조선 수군을 이끌어 국난을

In [6]:
# 모델에 응답 스키마를 추가
model_with_output = model.with_structured_output(OutputSchema)

res2 = model_with_output.invoke("이순신 장군에 대해 설명해줘.")

In [9]:
print(type(res2))
print(res2)
print(res2.is_real_person)
print(res2.description)

<class '__main__.OutputSchema'>
is_real_person=True description='이순신(1545–1598)은 조선 시대의 대표적인 군인이자 해군 장군으로 임진왜란(1592–1598) 기간 동안 탁월한 전략과 지휘력으로 일본의 해상 침략을 막아낸 인물입니다. 충청도 아산 출신으로 본관은 여양이며, 관직으로는 전라좌수사·선조 때의 해군 총지휘를 맡았습니다. 한산도 대첩(1592)에서 학익진(학의 날개) 전술로 큰 승리를 거두었고, 명량 해전(1597)에서는 13척의 배로 수십 배에 달하는 일본 함대를 격파하는 기적 같은 전과를 올렸습니다. 거북선(거북선)의 사용과 해전 전술 개선, 그리고 전투에서 한 번도 패하지 않은 기록으로 유명합니다. 그는 한때 모함(모함?)과 정치적 모함으로 관직에서 물러났다가 다시 복직하여 전투에 임했고, 1598년 노량해전에서 전사한 뒤 충무공(忠武公)이라는 시호를 받았습니다. 그의 난중일기(난중일기)는 전쟁과 군사 활동, 개인적 고뇌를 기록한 중요한 사료로 평가됩니다. 오늘날 이순신은 한국에서 충성과 용기의 상징으로 추앙받으며 기념비·동상·교육·문화콘텐츠 등으로 널리 알려져 있습니다.'
True
이순신(1545–1598)은 조선 시대의 대표적인 군인이자 해군 장군으로 임진왜란(1592–1598) 기간 동안 탁월한 전략과 지휘력으로 일본의 해상 침략을 막아낸 인물입니다. 충청도 아산 출신으로 본관은 여양이며, 관직으로는 전라좌수사·선조 때의 해군 총지휘를 맡았습니다. 한산도 대첩(1592)에서 학익진(학의 날개) 전술로 큰 승리를 거두었고, 명량 해전(1597)에서는 13척의 배로 수십 배에 달하는 일본 함대를 격파하는 기적 같은 전과를 올렸습니다. 거북선(거북선)의 사용과 해전 전술 개선, 그리고 전투에서 한 번도 패하지 않은 기록으로 유명합니다. 그는 한때 모함(모함?)과 정치적 모함으로 관직에서 물러났다가 다시 복직하여 전투에 임했고, 1598년 노량해전에서 전사한 뒤 충무공(忠武公)이라는 시호를

In [10]:
res3 = model_with_output.invoke("홍길동에 대해 설명해줘.")

In [11]:
res3

OutputSchema(is_real_person=False, description='홍길동은 고전소설 "홍길동전"의 주인공으로, 일반적으로 조선 중·후기 허균(許筠)에게서 유래한 작품으로 알려져 있다. 그는 서자로 태어나 신분 차별로 억압을 받지만, 뛰어난 재능과 의지로 부패한 권력과 사회적 불의를 바로잡기 위해 싸우는 인물로 그려진다. 작품은 신분제와 부패에 대한 비판, 사회 정의 실현을 주제로 하며 한국 문학사에서 대표적인 고전 소설로 평가받는다. 홍길동은 실제 인물이 아니라 허구적 인물이며, 이후 드라마·영화·만화 등 대중문화에서 의적(義賊)의 상징으로 널리 재창조되어 왔다.')

In [12]:
model.invoke("홍길동에 대해 설명해줘.")

AIMessage(content='간단히 말하면 홍길동은 조선 후기(16~17세기)에 나온 소설 《홍길동전》의 주인공으로, 한국문학에서 가장 유명한 민중영웅 중 하나입니다. 주요 내용과 특징을 요약하면 다음과 같습니다.\n\n- 작가·시기: 전통적으로 허균(許筠, 1569–1618)이 지은 작품으로 전해지며, 한글로 쓰인 근대 이전의 소설 가운데 대표작으로 꼽힙니다.\n- 줄거리(요약): 주인공 홍길동은 서자(첩의 아들)로서 신분 차별 때문에 관직 진출에 제한을 받고, 부패한 양반과 권력층에 맞서 도적 활동을 하며 민중을 돕습니다. 결국 자신의 세력을 모아 새 나라를 세우고 왕이 된다는 내용 등으로 전개됩니다.\n- 주제와 의미: 신분제와 부패한 사회에 대한 비판, 합법적 기회로 배제된 자의 저항, 정의와 평등에 대한 갈망을 다룹니다. 당시 사회·윤리 체계에 대한 도전적 메시지가 담겨 있어 진보적 문제 의식을 보여줍니다.\n- 문체·의의: 한글 소설로서 대중적 서사성과 풍자성이 강하고, 한국 근대문학으로 넘어가는 연결고리로 평가됩니다.\n- 문화적 영향: ‘한국의 로빈후드’로 비유되며, 수많은 영화·드라마·만화·연극·애니메이션 등으로 재창작되어 왔습니다. 또한 한국에서는 익명 예시 이름(우리나라의 John Doe)으로 ‘홍길동’을 쓰기도 합니다.\n\n더 자세한 줄거리(장별 요약), 원문 일부, 허균과의 관련 논쟁, 혹은 현대의 주요 각색 작품 목록을 원하시면 어떤 정보를 더 알고 싶은지 알려 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1147, 'prompt_tokens': 15, 'total_tokens': 1162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected

# Streaming 방식 응답 처리

- Streaming 방식 응답 처리: LLM이 텍스트를 모두 생성할 때까지 기다리지 않고, 생성되는 즉시 **부분적인 결과**를 실시간으로 전달받아 처리하는 방식을 의미. 이는 사용자가 응답을 더 빠르게 인지할 수 있게 해 주며, 특히 대화형 서비스, 실시간 UI 출력, 긴 문서 생성과 같은 상황에서 매우 유용하게 활용된다.

- `invoke()` 요청으로 받는 응답은 **비 스트리밍 방식**으로 모든 응답 텍스트 생성이 완료된 이후 그 결과를 한 번에 반환하는 구조이다. 반면 Streaming 방식은 **토큰(token) 단위 또는 여러 토큰이 묶인 청크(chunk) 단위**로 연속적인 데이터 스트림을 전송한다는 점에서 큰 차이가 있다. 즉, Streaming은 마치 사람이 타이핑을 치듯이 응답을 실시간으로 “흘려보내는” 방식이라고 이해할 수 있다.

- `모델.invoke(input, config)` → 응답 데이터
    - 모델이 전체 응답을 모두 생성한 뒤, 최종 결과를 한 번에 반환하는 방식이다.
    - 배치 처리나 후처리가 중요한 경우에 적합하다.
- `모델.stream(input, config)` → Iterator
    - 모델이 토큰을 생성하는 즉시, 순차적으로 결과를 제공하는 Iterator 형태로 반환한다.
    - 실시간 출력, 대화형 인터페이스, 웹 스트리밍 등에 특히 적합하다.

In [ ]:
model = ChatOpenAI(model="gpt-5-mini")   # , streaming=True)

iterator = model.stream("겨울에 낄 털장갑을 추천해줘. 추천하는 이유도 알려줘.")
print(type(iterator))

for token in iterator:
    print(token)

<class 'generator'>
content='' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b42-16e5-7f01-a591-1230abd214e8'
content='좋' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b42-16e5-7f01-a591-1230abd214e8'
content='아요' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b42-16e5-7f01-a591-1230abd214e8'
content=' —' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b42-16e5-7f01-a591-1230abd214e8'
content=' 겨' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b42-16e5-7f01-a591-1230abd214e8'
content='울' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b42-16e5-7f01-a591-1230abd214e8'
content='에' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b42-16e5-7f01-a591-1230abd214e8'
content=' 낄' additional_kwargs={} response_metadata={'model_provider

In [ ]:
for token in model.stream("겨울에 낄 털장갑을 추천해줘. 추천하는 이유도 알려줘."):
    print(token.content, end="")   # end="" : 한 토큰씩 나와 보기 힘드니 빈 문자열로 이어나오게 만듬.

좋아요 — 몇 가지 용도와 예산별로 실용적인 털장갑 추천과 추천 이유를 정리해 드릴게요. 먼저 한 가지만 알려주세요: 천연(동물)모피 허용하나요, 아니면 인조(페이크)털을 원하시나요? (모르면 둘 다 포함해서 추천드릴게요.)

1) 시어링(무스탕·양털) 라이닝 가죽 장갑 — 추천 이유
- 장점: 가죽 외피로 바람과 습기에 강하고, 안감인 시어링(양가죽/무스탕)의 보온성이 매우 높음. 통기성도 좋아 오래 착용해도 쾌적함. 외관이 고급스러워 데일리와 격식 모두 가능.
- 체크 포인트: 손바닥 그립(논슬립), 손가락 길이와 사이즈, 가죽 처리(발수코팅), 라이닝 두께.
- 예시 사용처: 출퇴근/외출, 포멀한 자리.

2) 퍼 라이닝(또는 퍼 커프) 니트/패브릭 장갑(인조털) — 추천 이유
- 장점: 가격대가 다양하고 스타일 선택 폭 넓음(컬러/패턴). 착용감 부드럽고 손목 부분 퍼로 보온성이 좋음. 세탁·관리 비교적 쉬움(인조털).
- 체크 포인트: 안감(플리스/기모 여부), 방풍성(바람 들어오지 않는지), 손목 길이.
- 예시 사용처: 일상/가볍게 외출할 때.

3) 기술성(방수·방풍) 글러브 + 퍼 트리밍 — 추천 이유
- 장점: 등산·스키·한파 출퇴근 등 실외 활동에 최적. 외피는 방수·방풍 소재, 안쪽은 보온 소재(예: Thinsulate)이고, 퍼 트리밍이 목 부분처럼 바람 차단에 도움.
- 체크 포인트: 방수 등급, 보온재(충전재 유형), 손목 조임/커프, 터치스크린 호환 여부.
- 예시 사용처: 야외활동·스키·자전거 출퇴근.

4) 미튼(벙어리장갑)형 퍼 라이닝 — 추천 이유
- 장점: 같은 두께의 장갑보다 보온성이 훨씬 좋음(손가락끼리 열을 공유). 내부가 퍼로 되어 있으면 극한의 추위에 유리.
- 체크 포인트: 손가락 움직임 제한, 내부 소재(털 두께), 손목 길이.
- 예시 사용처: 매우 추운 날씨, 장시간 야외 활동.

5) 터치스크린 호환 퍼 장갑(인조털 + 전도성 실) — 추천 이유
- 장갑 벗지 않고 스마트폰 사용 가능. 출퇴근·도시 생활에